In [ ]:
#@title Spearmint

# MongoDb
!apt install -y mongodb
!pip2.7 install pymongo
!mkdir mongodb
!mongod --fork --logpath mongolog --dbpath mongodb

# Spearmint
!pip2.7 install numpy==1.10
!pip2.7 install scipy==0.18.1
!pip2.7 install weave
!git clone https://github.com/HIPS/Spearmint.git
!cd Spearmint && git checkout PESC
!pip2.7 install -e Spearmint

In [ ]:
#@title Pycma

!pip install cma

In [ ]:
#@title GPflow

!pip install pip==18.1
!pip install git+https://github.com/GPflow/GPflowOpt.git --process-dependency-links

In [ ]:
#@title Base import and seed

import os
import math
import random
import numpy as np
import tensorflow as tf

import sys
if not 'emllib' in sys.path: sys.path.insert(1, 'emllib')

import pickle

def set_seed(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    tf.compat.v1.set_random_seed(seed)

# Run

In [ ]:
#@title PESC
!cd Spearmint/spearmint && python2 main.py ../examples/constrained

In [ ]:
#@title CMA-ES

import cma

es = cma.CMAEvolutionStrategy(8*[0], 0.5)
es.optimize(cma.ff.rosen, iterations=20)

(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 8 (seed=781024, Wed Jan 19 16:10:28 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 1.329413314625340e+02 1.0e+00 4.63e-01  4e-01  5e-01 0:00.0
    2     20 2.434802958991313e+01 1.2e+00 3.91e-01  4e-01  4e-01 0:00.0
    3     30 4.992302812686072e+01 1.4e+00 3.67e-01  3e-01  4e-01 0:00.0


In [ ]:
#@title cEI

import gpflow
import gpflowopt
import numpy as np

# Objective & constraint
def townsend(X):
    return -(np.cos((X[:,0]-0.1)*X[:,1])**2 + X[:,0] * np.sin(3*X[:,0]+X[:,1]))[:,None]

def constraint(X):
    return -(-np.cos(1.5*X[:,0]+np.pi)*np.cos(1.5*X[:,1])+np.sin(1.5*X[:,0]+np.pi)*np.sin(1.5*X[:,1]))[:,None]

# Setup input domain
domain = gpflowopt.domain.ContinuousParameter('x1', -2.25, 2.5) + \
         gpflowopt.domain.ContinuousParameter('x2', -2.5, 1.75)

# Plot
def plotfx():
    X = gpflowopt.design.FactorialDesign(101, domain).generate()
    Zo = townsend(X)
    Zc = constraint(X)
    mask = Zc>=0
    Zc[mask] = np.nan
    Zc[np.logical_not(mask)] = 1
    Z = Zo * Zc
    shape = (101, 101)

    f, axes = plt.subplots(1, 1, figsize=(7, 5))
    axes.contourf(X[:,0].reshape(shape), X[:,1].reshape(shape), Z.reshape(shape))
    axes.set_xlabel('x1')
    axes.set_ylabel('x2')
    axes.set_xlim([domain.lower[0], domain.upper[0]])
    axes.set_ylim([domain.lower[1], domain.upper[1]])
    return axes


AttributeError: ignored